In [ ]:
import pandas as pd

CSV_PATH = "smartphone_usage_time.csv"
TIME_COL = "Thoi gian trung binh su dung"

df = pd.read_csv(CSV_PATH, encoding="utf-8")

td = pd.to_timedelta(df[TIME_COL], errors="coerce")

hours = td.dt.total_seconds() / 3600
hours = hours.dropna()

sample_mean = hours.mean()
sample_variance_unbiased = hours.var(ddof=1)
n = hours.shape[0]
print(f"n = {n}")
print(f"Trung binh mau X = {sample_mean:.2f} gio")
print(f"Phuong sai mau hieu chinh Y = {sample_variance_unbiased:.2f} gio^2")


n = 265
Trung binh mau X = 6.39 gio
Phuong sai mau hieu chinh Y = 7.25 gio^2


In [4]:
def z_score(series: pd.Series):
    mu = series.mean()
    sigma = series.std(ddof=1)
    return (series - mu) / sigma
df['gio'] = td.dt.total_seconds() / 3600
valid = df['gio'].notna()
df.loc[valid, "gio_zscore"] = z_score(df.loc[valid, 'gio'])
df_fmt = df.copy()
df_out = df_fmt.head(5)
latex = df_out.to_latex(index=False, escape=False)  
with open("summary.tex", "w", encoding="utf-8") as f:
    f.write(latex)

In [5]:
import scipy.stats as st
import numpy as np
sample_std_unbiased = np.sqrt(sample_variance_unbiased)
standard_error_mean = sample_std_unbiased / np.sqrt(n)
ci_low, ci_high = st.t.interval(confidence=0.95, df=num_sample-1, loc=sample_mean, scale=standard_error_mean)
print(f"Khoang tin cay 95% cho ky vong: [{ci_low:.6f}, {ci_high:.6f}] gio")

NameError: name 'n' is not defined

In [41]:
mu0 = 7.3
alpha = 0.05

z_qs = ((sample_mean - mu0) / sample_std_unbiased) * np.sqrt(n)
p_value_z = st.norm.cdf(z_qs)
reject_z = p_value_z < alpha

print(f"z_qs = {z_qs:.6f}")
print(f"p-value (one-sided, H1: mu < {mu0}) = {p_value_z:.6e}")
print(f"Ket luan o muc y nghia {alpha*100:.0f}%: " + ("Bac bo H0" if reject_z else "Khong bac bo H0"))

res = st.ttest_1samp(hours, popmean=mu0, alternative="less")  # SciPy >= 1.9
t_qs = float(res.statistic)
p_value_t = float(res.pvalue)
reject_t = p_value_t < alpha

print(f"t_qs = {t_qs:.6f}")
print(f"p-value (one-sided, H1: mu < {mu0}) = {p_value_t:.6e}")
print(f"Ket luan o muc y nghia {alpha*100:.0f}%: " + ("Bac bo H0" if reject_t else "Khong bac bo H0"))


z_qs = -5.492302
p-value (one-sided, H1: mu < 7.3) = 1.983637e-08
Ket luan o muc y nghia 5%: Bac bo H0
t_qs = -5.492302
p-value (one-sided, H1: mu < 7.3) = 4.662221e-08
Ket luan o muc y nghia 5%: Bac bo H0
